## Trading Algorithm
***

#### Value at Risk (VaR) Trading Strategy
This code includes a simple historical VaR calculation (calculates VaR based on the last 30 days of price changes for wheat). If the VaR limit is exceeded (i.e. is less than the set limit, -3%), it doesn't enter a new position or exits the current position by triggering a sell action, thereby minimizing risk. 

#### Trading Algorithm - Code Overview

1. Downloads historical prices for wheat and corn futures.
2. Performs cointegration tests over 30-day windows.
3. Calculates Value at Risk (VaR) based on past 30-day price changes.
4. Buys wheat if cointegrated with corn and VaR is within limit.
5. Includes risk management using stop-loss and take-profit levels.
6. Sells position if stop-loss, take-profit, or VaR limit is triggered.
7. Outputs final capital.
8. It aims to trade based on statistical relationships while managing risks.

***

#### Install required libraries
`pip install yfinance statsmodels`

In [ ]:
import yfinance as yf
import numpy as np
import statsmodels.api as sm

In [ ]:
def calculate_var(price_changes, confidence_level=0.95):
    """Calculate Value at Risk (VaR) given price changes and confidence level"""
    return -np.percentile(price_changes, 100 * (1 - confidence_level))

In [ ]:
##-- Download historical data for wheat and corn futures
wheat_data = yf.download('ZW=F', start='2022-01-01', end='2022-12-31', interval='1d')
corn_data = yf.download('ZC=F', start='2022-01-01', end='2022-12-31', interval='1d')

##-- Close prices
wheat_prices = wheat_data['Close'].dropna().tolist()
corn_prices = corn_data['Close'].dropna().tolist()

##-- Initialize variables
capital = 1000
position = 0
entry_price = 0
window_size = 30
var_limit = -0.03  # -3% VaR limit

##-- Risk Management
stop_loss_percent = 0.02
take_profit_percent = 0.05

In [ ]:
for i in range(window_size, min(len(wheat_prices), len(corn_prices))):
    
    ##-- Cointegration test
    y = wheat_prices[i-window_size:i]
    X = corn_prices[i-window_size:i]
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    residuals = model.resid
    p_value = sm.tsa.adfuller(residuals)[1]

    ##-- Price changes for VaR
    price_changes = [wheat_prices[j] / wheat_prices[j-1] - 1 for j in range(i-window_size+1, i)]

    ##-- Calculate VaR
    current_var = calculate_var(price_changes)

    ##-- Buy if p-value is low (cointegrated)
    if p_value < 0.05 and position == 0 and current_var > var_limit:
        position = capital / wheat_prices[i]
        entry_price = wheat_prices[i]
        capital = 0
        print(f"Buy at {wheat_prices[i]}. Position: {position}")

    ##-- Risk Management
    if position > 0:
        if wheat_prices[i] <= entry_price * (1 - stop_loss_percent):
            capital = position * wheat_prices[i]
            print(f"Stop Loss Triggered. Sell at {wheat_prices[i]}. Capital: {capital}")
            position = 0
        elif wheat_prices[i] >= entry_price * (1 + take_profit_percent):
            capital = position * wheat_prices[i]
            print(f"Take Profit Triggered. Sell at {wheat_prices[i]}. Capital: {capital}")
            position = 0
        elif current_var <= var_limit:
            capital = position * wheat_prices[i]
            print(f"VaR Limit Exceeded. Sell at {wheat_prices[i]}. Capital: {capital}")
            position = 0

In [ ]:
##-- Final capital
if position > 0:
    capital = position * wheat_prices[-1]
print(f"Final Capital: {capital}")